# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
- In order to get the results based on sessionId and itemInSession, the table should have these 2 clumns as the composite Primary Key, which will be unique.

##### Create table

In [17]:
query = "DROP TABLE IF EXISTS session_activity"
session.execute(query)

query = "CREATE TABLE IF NOT EXISTS session_activity"
query = query + "(session_id int, item_in_session int, artist_name text, song_name text, length float, PRIMARY KEY(session_id, item_in_session))"
session.execute(query)

##### Insert value from 'event_datafile_new.csv' to the session_activity table

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_activity (session_id, item_in_session, artist_name, song_name, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

##### Run the query to answer question #1

In [19]:
query1 = "select artist_name, song_name, length from session_activity WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182¶
- In order to get the results based on userid and sessionId, the table should have userid and sessionId as the composite Primary Key, which will be unique.
- And in order to get the data sorted by itemInSession, itemInSession column should be set as the clustering key.

In [25]:
query = "DROP TABLE IF EXISTS user_activity"
session.execute(query)

query = "CREATE TABLE IF NOT EXISTS user_activity"
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_name text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
session.execute(query)

In [26]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_activity (user_id, session_id, item_in_session, artist_name, song_name, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]), line[0], line[9],line[1], line[4]))

In [29]:
query2 = "select artist_name, song_name, first_name, last_name from user_activity WHERE user_id = 10 and session_id = 182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- In order to get the results based on song, the table should have song and also user_id as the composite Primary Key, which will be unique.
- The user_id will be used in the Composite Primary Key as well because user's first and last name might not be unique.

In [30]:
query = "DROP TABLE IF EXISTS song_history"
session.execute(query)

query = "CREATE TABLE IF NOT EXISTS song_history"
query = query + "(song_name text, user_id int, first_name text, last_name text, PRIMARY KEY(song_name, user_id))"
session.execute(query)

In [31]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history (song_name, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [33]:
query3 = "select user_id, first_name, last_name from song_history WHERE song_name = 'All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_id, row.first_name, row.last_name)

29 Jacqueline Lynch
80 Tegan Levine
95 Sara Johnson


### Drop the tables before closing out the sessions

In [34]:
query = "drop table session_activity"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_activity"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()